In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import math
from scipy.optimize import basinhopping
#from scipy.optimize import dual_annealing
from math import *

# from pyOpt import Optimization
# from pyOpt import SLSQP

In [2]:
import scipy
scipy.__version__

'1.1.0'

In [37]:
## Get Non Marketing Values for each Week which can be directly used in Objective Function
path = "D:\\Affine_Projects\\WB_Warner_Bros\\MMM\\Optimisation\\22072019"
non_marketing_cols = ['const','Calender Week','Dec','Mar','Jul','1st week','2nd week','Is Action','Is Open World','same genre','different genre',
                     'Week Number','TubularViews']
title = "Lego Worlds"
#title = "Scribblenauts Showdown/Scribblenauts Fiesta"
channels = ["TVSpend","Facebook","Video","Display","Search","POSRebate","TradeDisplayPlacement"]
non_marketing_week_values = pd.read_excel(path+"\\non_marketing_week_data.xlsx",sheetname = "Data")
non_marketing_week_values["const"]=1
df_title_budget_constraints = pd.read_excel(path+"\\non_marketing_week_data.xlsx",sheetname = "Title_Budget_Constraints")
df_channel_ranges = pd.read_excel(path+"\\non_marketing_week_data.xlsx",sheetname = "Channel_Ranges")
df_weekwise_spending = pd.read_excel(path+"\\non_marketing_week_data.xlsx",sheetname = "Weekwise_Spending_Constraints")
marketing_beta = pd.read_excel(path+"\\non_marketing_week_data.xlsx",sheetname = "marketing_betaCoeff")

In [38]:
df_weekwise_spending = df_weekwise_spending[df_weekwise_spending["Title"]==title]
df_weekwise_spending

Title                Channel  \
7   Lego Worlds                TVSpend   
8   Lego Worlds               Facebook   
9   Lego Worlds                  Video   
10  Lego Worlds                 Search   
11  Lego Worlds                Display   
12  Lego Worlds              POSRebate   
13  Lego Worlds  TradeDisplayPlacement   

                                          Week Number  
7                                             4,5,6,7  
8                     0,1,2,3,4,5,6,33,34,35,37,38,39  
9               0,1,2,3,4,5,6,26,27,33,34,35,37,38,39  
10                                                NaN  
11                    0,1,2,3,4,5,6,33,34,35,37,38,39  
12  0,1,4,5,6,13,14,17,18,19,20,21,23,28,33,35,36,...  
13  0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,17,18,19,20...

In [39]:
### Uncomment this Only for Testing.
# non_marketing_week_values = non_marketing_week_values[non_marketing_week_values['Week Number']<=5]
# non_marketing_week_values

In [40]:
def get_title_budget(df_title_budget_constraints,title):
    return df_title_budget_constraints[df_title_budget_constraints["Title"]==title]["Budget"].values[0]

def get_Non_Marketing_Week_Values(non_marketing_week_values,non_marketing_cols,title):
    non_marketing_week_values = non_marketing_week_values[non_marketing_week_values["Title"]==title]
    non_marketing_week_values = non_marketing_week_values[non_marketing_cols]
    non_marketing_betaCoef = pd.read_excel(path+"\\non_marketing_week_data.xlsx",sheetname = "Beta_Coeff")
    non_marketing_week_beta_multiply = non_marketing_betaCoef.values*non_marketing_week_values
    non_marketing_week_exp = non_marketing_week_beta_multiply.apply(np.exp)
    non_marketing_week  = non_marketing_week_exp.product(axis=1)
    non_marketing = non_marketing_week.to_frame(name = "non_marketing")
    non_marketing = pd.concat([non_marketing,non_marketing_week_values[['Week Number']]],axis=1)
    return non_marketing


In [41]:
max_title_budget = get_title_budget(df_title_budget_constraints,title) 
max_title_budget

5410182

In [42]:
non_marketing = get_Non_Marketing_Week_Values(non_marketing_week_values.copy(),non_marketing_cols,title)
non_marketing

non_marketing  Week Number
219    4270.896397            0
220    2862.929906            1
221    3836.055868            2
222    2834.724406            3
223    2828.777731            4
224    2822.843530            5
225    2816.921779            6
226    2811.012450            7
227    2805.115518            8
228    2799.230956            9
229    2793.358739           10
230    2787.498841           11
231    2781.651236           12
232    2775.815897           13
233    2769.992800           14
234    2764.181919           15
235    3071.434514           16
236    3203.822797           17
237    3197.101829           18
238    3190.394960           19
239    3183.702160           20
240    2729.571729           21
241    2723.845643           22
242    2718.131569           23
243    2594.891543           24
244    2706.739357           25
245    2584.015853           26
246    2578.595114           27
247    2689.740501           28
248    2684.097973           29
249    2678.467281           30
250    2672.848402           31
251    2667.241310           32
252    2546.308647           33
253    2540.967010           34
254    2650.490510           35
255    2530.317328           36
256    2639.381795           37
257    4731.984739           38
258    4722.057998           39
259    4712.152081           40
260    4498.503208           41
261    4692.402545           42
262    2968.167387           43
263    2961.940776           44
264    2955.727227           45
265    2821.714619           46
266    2943.339205           47
267    2937.164679           48
268    2931.003105           49
269    2924.854457           50
270    3941.434412           51
271    3933.166085           52

In [43]:
non_marketing['non_marketing'].sum()

162818.7283193887

In [44]:
non_marketing_len = len(non_marketing['Week Number'])
print(non_marketing_len)

53


In [45]:
def roundoff_adstock(x):
    x = round(x,4)
    if(x==0):
        return 100
    else:
        #print("X is =>",x)
        log_10 = log10(x)
        floor_log10 = floor(log_10)
        lower_power10 = pow(10,floor_log10)
        first_digit = floor(x/lower_power10)
        final = first_digit * lower_power10
        return final


def add_single_adstock_col(df,col,decay_sat_par_val_dict,decay_param,saturation_param):
    df = df[[col]]
    decay = decay_sat_par_val_dict[decay_param]
    saturation = decay_sat_par_val_dict[saturation_param]
    df["pvals_"+col]=0
    df["adstock_"+col]=0
    for index in df.index:
        curr_val = df.loc[index,col]
        if(index==0):
            df.loc[index,"pvals_"+col] = 1- (1/exp((curr_val*saturation)/roundoff_adstock(curr_val)))
            df.loc[index,"adstock_"+col] = curr_val
        else:
            prev_pvals = df.loc[index-1,"pvals_"+col]
            df.loc[index,"pvals_"+col] = 1 - ((1-prev_pvals*exp(-1*decay))/exp((curr_val*saturation)/roundoff_adstock(curr_val)))
            df.loc[index,"adstock_"+col] = curr_val+(df.loc[index-1,"adstock_"+col]*df.loc[index,"pvals_"+col])
    df["ads_"+col+"_D"+str(decay_param)+"_S"+str(saturation_param)] = df["adstock_"+col]
    df = df.drop([col,"pvals_"+col,"adstock_"+col],axis=1)
    return df

# def add_single_adstock_col1(df,col,decay_sat_par_val_dict,decay_param,saturation_param):
#     print(col)
#     df = df[[col]]
#     decay = decay_sat_par_val_dict[decay_param]
#     saturation = decay_sat_par_val_dict[saturation_param]
#     df["adstock_"+col]=0
#     for index in df.index:
#         curr_val = df.loc[index,col]
#         if(index==0):
#             df.loc[index,"adstock_"+col] = curr_val
#         else:
#             df.loc[index,"adstock_"+col] = (1/(1+exp(-1*saturation*curr_val)))*curr_val + (decay* df.loc[index-1,"adstock_"+col]) 
#     df["ads_"+col+"_D"+str(decay_param)+"_S"+str(saturation_param)] = df["adstock_"+col]
#     df = df.drop([col,"adstock_"+col],axis=1)
#     print(df.columns)
#     return df 

decay_sat_par_val_dict = {
    1:0.0625,
    2:0.125,
    3:0.25,
    4:0.5,
    5:1,
    6:1.5,
    7:2,
    8:2.5,
    9:0.3,
    10:0.4,
    11:0.6,
    12:0.7,
    13:0.8,
    14:1.2,
    15:1.4
}

decay_params = {
    "TVSpend":{
        "decay":1,
        "saturation":8
    },
    "Facebook":{
        "decay":13,
        "saturation":7
    },
    "Video":{
        "decay":3,
        "saturation":4
    },
    "Display":{
        "decay":13,
        "saturation":15
    },
    "Search":{
        "decay":3,
        "saturation":4
    },
    "POSRebate":{
        "decay":3,
        "saturation":2
    },
    "TradeDisplayPlacement":{
        "decay":1,
        "saturation":5
    }
}


In [46]:
# starting_df = pd.read_excel(path+"\\non_marketing_week_data.xlsx",sheetname = "X0_1")
# starting_df = starting_df[starting_df.index<=(non_marketing_len-1)]
#print(non_marketing_len)
# x0 = np.array(starting_df.values).ravel()
x0 = np.array([1,1,1,1,1,1,1]*non_marketing_len)
x0

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [47]:
def get_marketing_series(x):
    l = len(x)
    TVSpend = pd.Series(x[0:l:7])
    Facebook =pd.Series(x[1:l:7])
    Video = pd.Series(x[2:l:7])
    Display = pd.Series(x[3:l:7])
    Search = pd.Series(x[4:l:7])
    POSRebate = pd.Series(x[5:l:7])
    TradeDisplayPlacement = pd.Series(x[6:l:7])
    return (TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement)

def get_marketing_df(TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement):
    df_TVSpend = TVSpend.to_frame(name = "TVSpend")
    df_Facebook = Facebook.to_frame(name = "Facebook")
    df_Video = Video.to_frame(name = "Video")
    df_Display = Display.to_frame(name="Display")
    df_Search = Search.to_frame(name="Search")
    df_POSRebate = POSRebate.to_frame(name="POSRebate")
    df_TradeDisplayPlacement = TradeDisplayPlacement.to_frame(name="TradeDisplayPlacement")
    return (df_TVSpend,df_Facebook,df_Video,df_Display,df_Search,df_POSRebate,df_TradeDisplayPlacement)


def get_ads_marketing(df_TVSpend,df_Facebook,df_Video,df_Display,df_Search,df_POSRebate,df_TradeDisplayPlacement,decay_sat_par_val_dict,decay_params):
    tvspend_ads = add_single_adstock_col(df_TVSpend,"TVSpend",decay_sat_par_val_dict,decay_params["TVSpend"]["decay"],decay_params["TVSpend"]["saturation"])
    facebook_ads = add_single_adstock_col(df_Facebook,"Facebook",decay_sat_par_val_dict,decay_params["Facebook"]["decay"],decay_params["Facebook"]["saturation"])
    video_ads = add_single_adstock_col(df_Video,"Video",decay_sat_par_val_dict,decay_params["Video"]["decay"],decay_params["Video"]["saturation"])
    display_ads = add_single_adstock_col(df_Display,"Display",decay_sat_par_val_dict,decay_params["Display"]["decay"],decay_params["Display"]["saturation"])
    search_ads = add_single_adstock_col(df_Search,"Search",decay_sat_par_val_dict,decay_params["Search"]["decay"],decay_params["Search"]["saturation"])
    posrebate_ads = add_single_adstock_col(df_POSRebate,"POSRebate",decay_sat_par_val_dict,decay_params["POSRebate"]["decay"],decay_params["POSRebate"]["saturation"])
    tradedisplayplacement_ads = add_single_adstock_col(df_TradeDisplayPlacement,"TradeDisplayPlacement",decay_sat_par_val_dict,decay_params["TradeDisplayPlacement"]["decay"],decay_params["TradeDisplayPlacement"]["saturation"])
    return (tvspend_ads[tvspend_ads.columns[0]],facebook_ads[facebook_ads.columns[0]],video_ads[video_ads.columns[0]],\
            display_ads[display_ads.columns[0]],search_ads[search_ads.columns[0]],posrebate_ads[posrebate_ads.columns[0]],\
            tradedisplayplacement_ads[tradedisplayplacement_ads.columns[0]])


In [48]:
def objective_function(x,non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,sign=-1.0):
    #print("Inside Objective Function x=> ",x)
    TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement = get_marketing_series(x)
    df_TVSpend,df_Facebook,df_Video,df_Display,df_Search,df_POSRebate,df_TradeDisplayPlacement = get_marketing_df(TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement)
    tvspend_ads,facebook_ads,video_ads,display_ads,search_ads,posrebate_ads,tradedisplayplacement_ads = get_ads_marketing(df_TVSpend,df_Facebook,df_Video,df_Display,df_Search,df_POSRebate,df_TradeDisplayPlacement,decay_sat_par_val_dict,decay_params)
    
    beta_tv = marketing_beta["TVSpend"].values[0]    
    beta_facebook = marketing_beta["Facebook"].values[0]
    beta_video = marketing_beta["Video"].values[0]
    beta_display = marketing_beta["Display"].values[0]
    beta_search = marketing_beta["Search"].values[0]
    beta_posrebate = marketing_beta["POSRebate"].values[0]
    beta_tradedisplayplacement = marketing_beta["TradeDisplayPlacement"].values[0]
    constant = non_marketing['non_marketing']
    constant = constant.reset_index()
    constant = constant['non_marketing']
    equation = constant * (tvspend_ads**beta_tv) * (facebook_ads**beta_facebook) * (video_ads**beta_video) * (display_ads**beta_display) * \
                  (search_ads**beta_search) * (df_POSRebate['POSRebate']**beta_posrebate) *\
                   (df_TradeDisplayPlacement['TradeDisplayPlacement']**beta_tradedisplayplacement)
    equation = np.asarray(list(map(int, equation)))
    
    return (sign * sum(equation))



def objective_functions_derivative(x,non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,sign=-1.0):
    TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement = get_marketing_series(x)
    df_TVSpend,df_Facebook,df_Video,df_Display,df_Search,df_POSRebate,df_TradeDisplayPlacement = get_marketing_df(TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement)
    tvspend_ads,facebook_ads,video_ads,display_ads,search_ads,posrebate_ads,tradedisplayplacement_ads = get_ads_marketing(df_TVSpend,df_Facebook,df_Video,df_Display,df_Search,df_POSRebate,df_TradeDisplayPlacement,decay_sat_par_val_dict,decay_params)
    
    beta_tv = marketing_beta["TVSpend"].values[0]    
    beta_facebook = marketing_beta["Facebook"].values[0]
    beta_video = marketing_beta["Video"].values[0]
    beta_display = marketing_beta["Display"].values[0]
    beta_search = marketing_beta["Search"].values[0]
    beta_posrebate = marketing_beta["POSRebate"].values[0]
    beta_tradedisplayplacement = marketing_beta["TradeDisplayPlacement"].values[0]
    constant = non_marketing['non_marketing']
    constant = constant.reset_index()
    constant = constant['non_marketing']
    
    tvspend_der  = constant*(beta_tv*(tvspend_ads**(beta_tv-1))) * (facebook_ads**beta_facebook) * (video_ads**beta_video) * (display_ads**beta_display) * \
                   (search_ads**beta_search) * (df_POSRebate['POSRebate']**beta_posrebate) *\
                   (df_TradeDisplayPlacement['TradeDisplayPlacement']**beta_tradedisplayplacement)
    tvspend_der  = np.asarray(list(tvspend_der))
            
    facebook_der = constant * (tvspend_ads**beta_tv) * (beta_facebook*(facebook_ads**(beta_facebook-1))) * (video_ads**beta_video) * (display_ads**beta_display) * \
                   (search_ads**beta_search) * (df_POSRebate['POSRebate']**beta_posrebate) *\
                   (df_TradeDisplayPlacement['TradeDisplayPlacement']**beta_tradedisplayplacement)
    facebook_der = np.asarray(list(facebook_der))
            
    video_der    = constant * (tvspend_ads**beta_tv) * (facebook_ads**beta_facebook) * (beta_video*(video_ads**(beta_video-1))) * (display_ads**beta_display) * \
                   (search_ads**beta_search) * (df_POSRebate['POSRebate']**beta_posrebate) *\
                   (df_TradeDisplayPlacement['TradeDisplayPlacement']**beta_tradedisplayplacement)
    video_der =    np.asarray(list(video_der))
    
    display_der  = constant * (tvspend_ads**beta_tv) * (facebook_ads**beta_facebook) * (video_ads**beta_video) * \
                   (beta_display*(display_ads**(beta_display-1))) *  (search_ads**beta_search) * (df_POSRebate['POSRebate']**beta_posrebate) *\
                   (df_TradeDisplayPlacement['TradeDisplayPlacement']**beta_tradedisplayplacement)
    display_der =  np.asarray(list(display_der))       
         
    search_der =   constant * (tvspend_ads**beta_tv) * (facebook_ads**beta_facebook) * (video_ads**beta_video) * (display_ads**beta_display) *\
                   (beta_search*(search_ads**(beta_search-1))) * (df_POSRebate['POSRebate']**beta_posrebate) *\
                   (df_TradeDisplayPlacement['TradeDisplayPlacement']**beta_tradedisplayplacement)
    search_der =    np.asarray(list(search_der))
    
    posrebate_der = constant * (tvspend_ads**beta_tv) * (facebook_ads**beta_facebook) * (video_ads**beta_video) * (display_ads**beta_display) *\
                    (search_ads**beta_search) * (beta_posrebate*(df_POSRebate['POSRebate']**(beta_posrebate-1))) *\
                    (df_TradeDisplayPlacement['TradeDisplayPlacement']**beta_tradedisplayplacement)
    posrebate_der = np.asarray(list(posrebate_der)) 
            
    tradedisplayplacement_der = constant * (tvspend_ads**beta_tv) * (facebook_ads**beta_facebook) * (video_ads**beta_video) * (display_ads**beta_display) *\
                    (search_ads**beta_search) * (df_POSRebate['POSRebate']**beta_posrebate) *\
                    (beta_tradedisplayplacement*(df_TradeDisplayPlacement['TradeDisplayPlacement']**(beta_tradedisplayplacement-1)))
    tradedisplayplacement_der = np.asarray(list(tradedisplayplacement_der))
    
    dfdx0 = np.column_stack(((sign*tvspend_der),(sign*facebook_der),(sign*video_der),(sign*display_der),(sign*search_der),(sign*posrebate_der),(sign*tradedisplayplacement_der))).ravel()
    return dfdx0


In [49]:
objective_functions_derivative(x0,non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,sign=-1.0)

array([-1.45210478e+01, -1.07199500e+02, -1.28553982e+02, -1.62721153e+02,
       -2.09273923e+01, -2.64795577e+01, -3.73703435e+02, -5.18608243e+00,
       -3.97123163e+01, -5.78125902e+01, -6.29176835e+01, -9.41135189e+00,
       -1.88070556e+01, -2.65422155e+02, -4.79430290e+00, -3.81118157e+01,
       -6.15295895e+01, -6.26178887e+01, -1.00164448e+01, -2.60272194e+01,
       -3.67319628e+02, -2.72102799e+00, -2.24425313e+01, -3.88627154e+01,
       -3.81733470e+01, -6.32648855e+00, -1.96536166e+01, -2.77369589e+02,
       -2.21157391e+00, -1.89134579e+01, -3.46905645e+01, -3.32593610e+01,
       -5.64730120e+00, -1.99206451e+01, -2.81138136e+02, -1.86538196e+00,
       -1.65312435e+01, -3.19540534e+01, -3.00106029e+01, -5.20182265e+00,
       -2.01133685e+01, -2.83858023e+02, -1.61413614e+00, -1.48146752e+01,
       -3.00990282e+01, -2.77228497e+01, -4.89984179e+00, -2.02539612e+01,
       -2.85842194e+02, -1.42316073e+00, -1.35198281e+01, -2.88115325e+01,
       -2.60399064e+01, -

In [50]:
print(objective_function(x0,non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,sign=-1.0))

-194075.0


In [51]:
df_title_budget_constraints

Title   Budget
0                             Lego Incredibles  8257208
1                     Lego Marvel Super Hero 2  5227928
2                 Lego Ninjago Movie Videogame  6616255
3                                Lego Super DC  7066587
4                                  Lego Worlds  5410182
5                         Cars-3-Driven-to-Win  3142104
6  Scribblenauts Showdown/Scribblenauts Fiesta  1274396

In [52]:
def title_budget_constraint(x,budget):
    max_title_spend = budget
    max_title_spend = max_title_spend - sum(x)
    return max_title_spend

def get_start_index(channel):
    s_index = 0
    if(channel=="TVSpend"): s_index=0
    elif(channel=="Facebook"): s_index = 1
    elif(channel=="Video"): s_index=2
    elif(channel=="Display"): s_index =3
    elif(channel == "Search"): s_index=4
    elif(channel =="POSRebate"): s_index=5
    else: s_index=6
    return s_index
 
def channel_constraint(x,constraint_range,df_channel_ranges,channel,title_budget,non_marketing_len):
    s_index = get_start_index(channel)
    channel_sum = 0
    for i in range(0,non_marketing_len):
        channel_sum = channel_sum + x[s_index + (i*7)]
    
    min_channel_range_percentage = df_channel_ranges[df_channel_ranges["Channel"]==channel]["Min_Percentage"].values[0]
    max_channel_range_percentage = df_channel_ranges[df_channel_ranges["Channel"]==channel]["Max_Percentage"].values[0]
    if(constraint_range=="Min_Percentage"):
        return channel_sum - ((min_channel_range_percentage/100)*title_budget)
    else:
        return ((max_channel_range_percentage/100)*title_budget) - channel_sum   
    

    

In [17]:
### Added this so that we can restrict spending (bounds) for the given weeks.

def isNaN(num):
    return num != num


def bounds_week_spending(title,channel,non_marketing_len,bounds):
    arr = bounds.copy()
    s_index = get_start_index(channel)
    spend_weeks = df_weekwise_spending[df_weekwise_spending["Channel"]==channel]["Week Number"].values[0]
    non_zero_weeks = []
    if(isNaN(spend_weeks)):
        non_zero_weeks = []
    elif(type(spend_weeks)==int):
        non_zero_weeks = [spend_weeks]
    else:
        non_zero_weeks = list(map(int,spend_weeks.split(",")))

    all_weeks = [i for  i in range(0,non_marketing_len)]
    zero_weeks = [x for x in all_weeks if x not in non_zero_weeks]
    for i in range(0,non_marketing_len):
        if(i in zero_weeks):
            l = list(arr[s_index + (i*7)])
            l[1] = 1
            arr[s_index + (i*7)] = tuple(l)
    return arr

bounds = [(1.0,1000000)]*non_marketing_len*7

for channel in channels:
    print(channel)
    bounds = bounds_week_spending(title,channel,non_marketing_len,bounds)

bounds


TVSpend
Facebook
Video
Display
Search
POSRebate
TradeDisplayPlacement


[(1.0, 1),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1000000),
 (1.0, 1),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1000000),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1000000),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1000000),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 1),
 (1.0, 

In [18]:
channel_constraint(x0,"Max_Percentage",df_channel_ranges,"POSRebate",max_title_budget,non_marketing_len)

4057583.5

In [19]:
def get_min_max_channel_constraints(channels,df_channel_ranges,max_title_budget,non_marketing_len):
    min_channel_constraints = []
    max_channel_constraints = []
    for channel in channels:
        dic_channel_min_constraint = {
            "type":"ineq",
            "fun" :channel_constraint,
            "args":["Min_Percentage",df_channel_ranges,channel,max_title_budget,non_marketing_len]
        }
        dic_channel_max_constraint = {
            "type":"ineq",
            "fun" :channel_constraint,
            "args":["Max_Percentage",df_channel_ranges,channel,max_title_budget,non_marketing_len]
        }
        min_channel_constraints.append(dic_channel_min_constraint)
        max_channel_constraints.append(dic_channel_max_constraint)
    return (min_channel_constraints,max_channel_constraints)

In [20]:
def get_title_budget_constraint(budget):
    dic_title_budget_constraint = {}
    dic_title_budget_constraint.clear()
    #print("dic_title_budget_constraint = ",dic_title_budget_constraint)
    dic_title_budget_constraint["type"] = "eq"
    dic_title_budget_constraint["fun"] = title_budget_constraint
    dic_title_budget_constraint["args"] = [budget]
    return dic_title_budget_constraint


In [21]:
title_budget_const = get_title_budget_constraint(max_title_budget.copy())
min_channel_constraints,max_channel_constraints  = get_min_max_channel_constraints(channels,df_channel_ranges,max_title_budget,non_marketing_len)
constraints = []
constraints.append(title_budget_const)
constraints = constraints + min_channel_constraints
constraints = constraints + max_channel_constraints

In [22]:
get_title_budget_constraint(max_title_budget.copy())

{'args': [5410182],
 'fun': <function __main__.title_budget_constraint>,
 'type': 'eq'}

In [23]:
title_budget_const
#constraints
#len(min_channel_constraints)

{'args': [5410182],
 'fun': <function __main__.title_budget_constraint>,
 'type': 'eq'}

In [24]:
# constraints = [dic_title_budget_constraint]
# constraints = constraints + min_channel_constraints
# constraints = constraints + max_channel_constraints

In [25]:
Nfeval= 0 
def callbackF(Xi):
    global Nfeval
    print("Nfeval = {}  {}   {}   {}".format(Nfeval, Xi[0], Xi[1], Xi[2]))
    Nfeval += 1

In [32]:
def get_overallChannelSpend(result,channels,columns,budget):
    TVSpend,Facebook,Video,Display,Search,POS,TradeDisplay = get_marketing_series(result.x)
    df_result = pd.concat([TVSpend,Facebook,Video,Display,Search,POS,TradeDisplay],axis=1)
    df_result.columns = columns
    df_1 = df_result.sum(axis=0).to_frame(name=budget).transpose()
    return (df_1,df_result)


# Running Optimiser

In [26]:
from datetime import datetime
s_time = datetime.now().timestamp()
result = minimize(objective_function,x0,args = (non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,-1),method = "SLSQP" ,bounds = bounds, constraints = constraints,jac = objective_functions_derivative,callback = callbackF,options = {"maxiter":40,"disp":True},tol=.001)
e_time = datetime.now().timestamp()

print("s_time = ",s_time)
print("e_time = ",e_time)
print("sec = ",(e_time-s_time))

Nfeval = 0  1.0   29211.234159818   46971.123254510596
Nfeval = 1  1.0   29212.20071133889   46973.23599240262
Nfeval = 2  1.0   29212.21946574681   46973.24792971505
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -1011618.0
            Iterations: 3
            Function evaluations: 5
            Gradient evaluations: 3
s_time =  1565686663.125443
e_time =  1565686713.728778
sec =  50.60333490371704


In [36]:
columns = ["TVSpend","Facebook","Video","Display","Search","POSRebate","TradeDisplayPlacement"]
df_2,df_basin_result = get_overallChannelSpend(result,7,columns,5410182)
df_2

TVSpend       Facebook          Video        Display  Search  \
5410182  811527.3  378712.740001  703323.660001  811527.300001    53.0   

            POSRebate  TradeDisplayPlacement  
5410182  1.623002e+06           1.082036e+06

### For Running Multiple times to get the best result

In [208]:
import random
def get_starting_point(seed,channels,non_marketing_len,x_prev):
    random.seed(seed)
    xi = []
    for i in range(0,(len(channels)*non_marketing_len)):
        xi.append(random.randint(100000,600000))
    return np.array(xi)

def callbackF(Xi):
    global Nfeval
    print("Nfeval = {}  {}   {}   {}".format(Nfeval, Xi[0], Xi[1], Xi[2]))
    Nfeval += 1
    
noOfIterations = 15
results = []
x_in = x0
Nfeval= 0 
s_time = datetime.now().timestamp()
for i in range(0,noOfIterations):
    print("Iteration is ===========>>>>",i)
    result = minimize(objective_function,x_in,args = (non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,-1),method = "SLSQP" ,bounds = bounds, constraints = constraints,jac = objective_functions_derivative,callback = callbackF,options = {"maxiter":40,"disp":True},tol=.001)
    print("result for iteration {} = {}".format(i,result.fun*-1))
    results.append(result)
    x_in = get_starting_point(result.fun*-1,channels,non_marketing_len,result.x)
    Nfeval= 0 
e_time = datetime.now().timestamp()

print("sec is =>>",(e_time-s_time))


Iteration is ===========>>>> 0
Nfeval = 0  247878.25813484343   256607.32510699733   455805.5544982495
Nfeval = 1  247878.59145110706   256607.35398718438   455805.47575449804
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -630177.0
            Iterations: 2
            Function evaluations: 3
            Gradient evaluations: 2
result for iteration 0 = 630177.0
Iteration is ===========>>>> 1
Nfeval = 0  97817.17361810291   186200.1774965365   274673.3780292907
Nfeval = 1  97759.7817899177   186142.78962884826   274599.2218077292
Nfeval = 2  97757.8357080327   186140.84795065975   274596.02927150467
Nfeval = 3  97757.89900554251   186140.91604323743   274595.5875148791
Nfeval = 4  97758.0696587624   186141.10459459285   274594.29368399037
Nfeval = 5  97758.03833768386   186141.10300867038   274592.346538747
Nfeval = 6  97757.83094927183   186140.96382797734   274587.8729735045
Nfeval = 7  97757.32244523193   186140.57257983665   274580.344877

In [179]:
max_result = results[0].fun*-1
result_selected = results[0]
for i in range(1,noOfIterations):
    if((results[i].fun*-1) >=max_result):
        max_result = results[i].fun*-1
        result_selected = results[i]

result_selected  

     fun: -630339.0
     jac: array([-9.62122230e-04, -6.86110975e-03, -4.63208543e-03, -2.67248123e+03,
       -2.50613415e-03, -4.34891959e+02, -5.05651910e-02, -4.20559424e-04,
       -4.50747219e-03, -3.19466062e-03, -1.14317560e+03, -2.13740556e-03,
       -3.41712630e+02, -5.45663319e-02, -2.32791891e-04, -3.07657590e-03,
       -2.10376486e-03, -6.86179203e+02, -1.92492532e-03, -2.85211416e+02,
       -3.64665008e-02, -1.30069577e-04, -2.07209899e-03, -1.92172900e-03,
       -3.98219984e+02, -1.40665213e-03, -4.26572851e-03, -2.74577400e-02,
       -8.01380668e-05, -1.76134715e-03, -1.90468892e-03, -2.68281240e+02,
       -1.09765361e-03, -3.34605152e-03, -2.21504440e-02, -7.67109991e-05,
       -1.90507098e-03, -2.38790355e-03, -2.38765030e+02, -1.08172424e-03,
       -9.53205350e-04, -2.22764656e-02, -1.29540688e-04, -3.62941752e-03,
       -5.24666054e-03, -3.84178056e+02, -1.87627104e-03, -1.41842598e-03,
       -4.02638720e-02, -1.21428102e-04, -3.41933107e-03, -6.69256817e

In [180]:
TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement = get_marketing_series(result_selected.x)
df_result = pd.concat([TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement],axis=1)
df_result.columns = ["TVSpend","Facebook","Video","Display","Search","POSRebate","TradeDisplayPlacement"]
df_result

TVSpend       Facebook          Video  Display         Search  \
0   247878.591454  256607.353987  455805.475754      1.0  137145.591226   
1   197878.590268   66607.349400  183805.472904      1.0   40145.590695   
2   227878.589851   91607.346480  255805.470761      1.0   25146.590967   
3   192878.589613   46607.344319    5806.470337      1.0   25146.590162   
4   237878.589505       1.000000    5806.470394      1.0   25146.590020   
5    47879.589507       1.000000    5806.471676      1.0   25146.591358   
6    47879.589633       1.000000    5806.478179      1.0   25146.598259   
7    47879.589620   16608.347784       1.000000      1.0   25146.601365   
8        1.000000   16608.348038       1.000000      1.0   25146.605964   
9        1.000000       1.000000       1.000000      1.0       1.000000   
10       1.000000       1.000000       1.000000      1.0       1.000000   
11       1.000000       1.000000       1.000000      1.0       1.000000   
12       1.000000       1.000000       1.000000      1.0       1.000000   
13       1.000000       1.000000       1.000000      1.0       1.000000   
14       1.000000       1.000000       1.000000      1.0       1.000000   
15       1.000000       1.000000       1.000000      1.0       1.000000   
16       1.000000       1.000000       1.000000      1.0       1.000000   
17       1.000000       1.000000       1.000000      1.0       1.000000   
18       1.000000       1.000000       1.000000      1.0       1.000000   
19       1.000000       1.000000       1.000000      1.0       1.000000   
20       1.000000       1.000000       1.000000      1.0       1.000000   

        POSRebate  TradeDisplayPlacement  
0        1.000000          121379.900354  
1        1.000000           88379.918466  
2        1.000000          110379.869262  
3    48063.515018          105379.846336  
4    48023.147715          102379.834559  
5   167878.591553          101379.836748  
6   197878.592521           98379.879599  
7   160878.593301          100379.874385  
8   158378.593371           90379.887028  
9   156278.593198           95379.874539  
10  237878.591749          150379.838231  
11  227878.590930          155379.816987  
12  224878.589926               1.000000  
13  213878.589951               1.000000  
14  217878.589886               1.000000  
15  154878.590174               1.000000  
16   83208.591472               1.000000  
17   82348.592391           31097.587186  
18   79758.591536               1.000000  
19   80708.593438           31310.729187  
20   97878.592490           31344.707162

In [28]:
### For Basin Hopping
minimizer_kwargs = {"method": "SLSQP","constraints":constraints,"bounds":bounds,"args":(non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,-1),"jac":objective_functions_derivative}

basin_result = basinhopping(objective_function, x0, minimizer_kwargs=minimizer_kwargs,niter=10,
            disp=True,stepsize=0.001,niter_success=5,seed = 7)

basinhopping step 0: f -726195
basinhopping step 1: f -737432 trial_f -737432 accepted 1  lowest_f -737432
found new global minimum on step 1 with function value -737432
basinhopping step 2: f -758315 trial_f -758315 accepted 1  lowest_f -758315
found new global minimum on step 2 with function value -758315
basinhopping step 3: f -761897 trial_f -761897 accepted 1  lowest_f -761897
found new global minimum on step 3 with function value -761897
basinhopping step 4: f -789125 trial_f -789125 accepted 1  lowest_f -789125
found new global minimum on step 4 with function value -789125
basinhopping step 5: f -789364 trial_f -789364 accepted 1  lowest_f -789364
found new global minimum on step 5 with function value -789364
basinhopping step 6: f -789434 trial_f -789434 accepted 1  lowest_f -789434
found new global minimum on step 6 with function value -789434
basinhopping step 7: f -790423 trial_f -790423 accepted 1  lowest_f -790423
found new global minimum on step 7 with function value -790

In [29]:
basin_result

                        fun: -790511.0
 lowest_optimization_result:      fun: -790511.0
     jac: array([-4.89740455e-04, -2.66196337e-03, -2.31001111e-03, -2.51081681e-03,
       -1.52266349e-03, -2.17612619e+02, -3.38460085e-02, -1.79735982e-04,
       -1.68182274e-03, -1.35651085e-03, -1.28618636e-03, -1.22256019e-03,
       -1.43051526e+02, -3.52311251e-02, -1.73710260e-04, -2.01838770e-03,
       -1.64909540e-03, -2.08293232e-03, -2.36827546e-03, -1.98845718e+02,
       -3.51591080e-02, -1.27063204e-04, -1.83709944e-03, -2.09679612e-03,
       -2.32157130e-03, -3.05793197e-03, -1.88052886e+02, -3.55875663e-02,
       -9.92942646e-05, -1.89822862e-03, -2.68627245e-03, -2.61411212e-03,
       -3.94598435e-03, -1.79441190e+02, -3.54917258e-02, -9.78541059e-05,
       -2.02133883e-03, -3.53417390e-03, -3.02519260e-03, -5.20907094e-03,
       -1.49363122e-03, -3.54293338e-02, -9.31158850e-05, -2.07801785e-03,
       -4.48679493e-03, -3.37934099e-03, -6.62350873e-03, -1.49008657e-03,
  

In [31]:
TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement = get_marketing_series(basin_result.x)
df_result = pd.concat([TVSpend,Facebook,Video,Display,Search,POSRebate,TradeDisplayPlacement],axis=1)
df_result.columns = ["TVSpend","Facebook","Video","Display","Search","POSRebate","TradeDisplayPlacement"]
df_result

TVSpend       Facebook          Video        Display         Search  \
0   243671.828594  330951.254515  457345.923204  532601.436939  112949.556665   
1   192994.318179   26890.017543  178460.828772  179141.829090   14981.984444   
2   191500.395876   76507.401929  205694.947231       1.000000       1.000000   
3   184255.555150   28689.660473       1.000000       1.000000       1.000000   
4   179968.150595       1.000000       1.000000       1.000000       1.000000   
5        1.000000       1.000000       1.000000       1.000000       1.000000   
6        1.000000       1.000000       1.000000       1.000000       1.000000   
7        1.000000       1.000000       1.000000       1.000000       1.000000   
8        1.000000       1.000000       1.000000       1.000000       1.000000   
9        1.000000       1.000000       1.000000       1.000000       1.000000   
10       1.000000       1.000000       1.000000       1.000000       1.000000   
11       1.000000       1.000000       1.000000       1.000000       1.000000   
12       1.000000       1.000000       1.000000       1.000000       1.000000   
13       1.000000       1.000000       1.000000       1.000000       1.000000   
14       1.000000       1.000000       1.000000       1.000000       1.000000   
15       1.000000       1.000000       1.000000       1.000000       1.000000   
16       1.000000       1.000000       1.000000       1.000000       1.000000   
17       1.000000       1.000000       1.000000       1.000000       1.000000   
18       1.000000       1.000000       1.000000       1.000000       1.000000   
19       1.000000       1.000000       1.000000       1.000000       1.000000   
20       1.000000       1.000000       1.000000       1.000000       1.000000   
21       1.000000       1.000000       1.000000       1.000000       1.000000   
22       1.000000       1.000000       1.000000       1.000000       1.000000   
23       1.000000       1.000000       1.000000       1.000000       1.000000   
24       1.000000       1.000000       1.000000       1.000000       1.000000   
25       1.000000       1.000000       1.000000       1.000000       1.000000   
26       1.000000       1.000000       1.000000       1.000000       1.000000   
27       1.000000       1.000000       1.000000       1.000000       1.000000   
28       1.000000       1.000000       1.000000       1.000000       1.000000   
29       1.000000       1.000000       1.000000       1.000000       1.000000   
30       1.000000       1.000000       1.000000       1.000000       1.000000   
31       1.000000       1.000000       1.000000       1.000000       1.000000   
32       1.000000       1.000000       1.000000       1.000000       1.000000   
33       1.000000       1.000000       1.000000       1.000000       1.000000   
34       1.000000       1.000000       1.000000       1.000000       1.000000   
35       1.000000       1.000000       1.000000       1.000000       1.000000   
36       1.000000       1.000000       1.000000       1.000000       1.000000   
37       1.000000       1.000000       1.000000       1.000000       1.000000   
38       1.000000       1.000000       1.000000       1.000000       1.000000   
39       1.000000       1.000000       1.000000       1.000000       1.000000   
40       1.000000       1.000000       1.000000       1.000000       1.000000   
41       1.000000       1.000000       1.000000       1.000000       1.000000   
42       1.000000       1.000000       1.000000       1.000000       1.000000   
43       1.000000       1.000000       1.000000       1.000000       1.000000   
44       1.000000       1.000000       1.000000       1.000000       1.000000   
45       1.000000       1.000000       1.000000       1.000000       1.000000   
46       1.000000       1.000000       1.000000       1.000000       1.000000   
47       1.000000       1.000000       1.000000       1.000000       1.000000   
48       1.000000       1.000000       1.

In [29]:
# from datetime import datetime
# Nfeval= 0 
# def callbackBasin(X,F,accept):
#     global Nfeval
#     print("Nfeval = {}  {}   {}   {}".format(Nfeval,F,accept))
#     Nfeval += 1
    
# def take_step(X):
#     return 25000

# s_time = datetime.now()
# print("s_time = ",s_time)
# minimizer_kwargs = {"method": "SLSQP","constraints":constraints,"bounds":bounds,"args":(non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,-1),"jac":objective_functions_derivative}

# basin_result = basinhopping(objective_function, x0, minimizer_kwargs=minimizer_kwargs,niter=10,
#             disp=True,stepsize=0.001,niter_success=2,seed = 7,callback = callbackBasin)
# e_time = datetime.now()
# print("e_time -===",e_time)
# print("Time is ----",(e_time-s_time))

s_time =  2019-08-01 16:32:32.579213
basinhopping step 0: f -726195


KeyboardInterrupt: 

### For Multiple Iterations with different step sized of Budget

In [151]:
def callbackF(Xi):
    global Nfeval
    print("Nfeval = {}  {}   {}   {}".format(Nfeval, Xi[0], Xi[1], Xi[2]))
    Nfeval += 1

budget_units = {}
ranges = [(225000,250000,25000)]
for rangei in ranges:
    print("--Rangei is ===>>>",rangei)
    for budget in range(rangei[0],rangei[1],rangei[2]):
        print("---------Budget is------------ >>>>",budget)
        title_budget_const = get_title_budget_constraint(budget)
        min_channel_constraints,max_channel_constraints  = get_min_max_channel_constraints(channels,df_channel_ranges,budget,non_marketing_len)
        constraints = []
        constraints.append(title_budget_const)
        constraints = constraints + min_channel_constraints
        constraints = constraints + max_channel_constraints
        Nfeval= 0 
        result_budget = minimize(objective_function,x0,args = (non_marketing,marketing_beta,decay_sat_par_val_dict,decay_params,-1),method = "SLSQP" ,bounds = bounds, constraints = constraints,jac = objective_functions_derivative,callback = callbackF,options = {"maxiter":40,"disp":True},tol=.001)
        print(result_budget.fun)
        budget_units[budget] = (result_budget.fun * -1)
        Nfeval = 0

--Rangei is ===>>> (225000, 250000, 25000)
---------Budget is------------ >>>> 225000
Nfeval = 0  11241.437717535016   3220.7960631179685   4371.851109615367
Nfeval = 1  11241.445961132242   3220.8870964926646   4371.941247845381
Nfeval = 2  11241.458587888455   3221.3352120713694   4372.38189746758
Nfeval = 3  11241.521666781633   3223.5752223159716   4374.584643903783
Nfeval = 4  11241.836701404804   3234.7602601134363   4385.583918616313
Nfeval = 5  11243.4012816119   3290.3071648763153   4440.209784369135
Nfeval = 6  11250.969929036553   3559.00634724777   4704.4607333269605
Nfeval = 7  11283.7070283154   4721.181646794042   5847.425345608879
Nfeval = 8  11300.458646209494   5315.818144280785   6432.268088062477
Nfeval = 9  11300.461674261025   5315.878900023042   6432.362307857805
Nfeval = 10  11300.473688513512   5316.117522022836   6432.7348741343185
Nfeval = 11  11300.53393140367   5317.318274181728   6434.605358130054
Nfeval = 12  11300.910244786433   5324.820006391543   6446.

In [149]:
budget_units

{225000: 321971.0,
 250000: 316526.0,
 275000: 318344.0,
 300000: 320431.0,
 325000: 328800.0,
 350000: 324340.0,
 375000: 325290.0,
 400000: 334269.0,
 425000: 338193.0,
 450000: 330071.0,
 475000: 331455.0,
 500000: 332909.0,
 525000: 334057.0,
 550000: 342951.0,
 575000: 336378.0,
 600000: 337823.0,
 625000: 339204.0,
 650000: 348060.0,
 675000: 341564.0,
 700000: 342818.0,
 725000: 343771.0,
 750000: 344952.0,
 775000: 346087.0,
 800000: 347191.0,
 825000: 348250.0,
 850000: 349284.0,
 875000: 350289.0,
 900000: 351265.0,
 925000: 352210.0,
 950000: 362052.0,
 975000: 354025.0}

In [150]:
budget_df = pd.DataFrame.from_dict(budget_units,orient='index')
budget_df.columns = ['Units']
budget_df

Units
600000  337823.0
500000  332909.0
975000  354025.0
850000  349284.0
325000  328800.0
450000  330071.0
275000  318344.0
425000  338193.0
575000  336378.0
400000  334269.0
375000  325290.0
950000  362052.0
875000  350289.0
350000  324340.0
300000  320431.0
250000  316526.0
900000  351265.0
775000  346087.0
625000  339204.0
825000  348250.0
225000  321971.0
800000  347191.0
550000  342951.0
525000  334057.0
750000  344952.0
725000  343771.0
475000  331455.0
700000  342818.0
925000  352210.0
675000  341564.0
650000  348060.0